In [4]:
import pandas as pd
import numpy as np
from tabulate import tabulate
from prettytable import PrettyTable
from termcolor import cprint
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from plotly.offline import iplot
import plotly.graph_objs as go
import matplotlib.pyplot as plt

In [1]:
#%% FUNCTION FOR FORWARD FEATURE SELECTION

def forward_selection(df):
    from sklearn.feature_selection import f_regression
    df=df.fillna(df.mean())
    le = LabelEncoder()
    cat_cols=list(df.select_dtypes('object').columns)
    for fet in cat_cols:
        #le.fit(df[fet])
        df[fet] = le.fit_transform(df[fet].astype(str))
    target=input('ENTER THE TARGET VARIABLE : ')
    Target=df[target]
    X=df.drop(columns=target)
    ffs = f_regression(X,Target)
    t = Table([X.columns, ffs[0], ffs[1]], names=('FEATURES', 'f-Value', 'p-Value'))
    print(t)
    #print(ffs)
    #data = {'names': ffs[0], 'values': ffs[1]}
    #df1 = pandas.DataFrame(data=data)
    #print(df1)
    variable = [ ]
    threshold=int(input("Enter f_value : (integer value ranging from 10 to 80)"))
    for i in range(0,len(df.columns)-1):
        if ffs[0][i] >=threshold:
            variable.append(df.columns[i])
    return variable     ##this is list of accepted variables

In [2]:
def backward_elimination(df):
    import statsmodels.api as sm
    df=df.fillna(df.mean())
    le = LabelEncoder()
    cat_cols=list(df.select_dtypes('object').columns)
    for fet in cat_cols:
        #le.fit(df[fet])
        df[fet] = le.fit_transform(df[fet].astype(str))
    features = df.columns.tolist()
    target=input('ENTER THE TARGET VARIABLE : ')
    Target=df[target]
    X=df.drop(columns=target)
    threshold=float(input("Choose a significance level 0.01 to 0.05"))
    while(len(features)>0):
        features_with_constant = sm.add_constant(df[features])
        p_values = sm.OLS(Target, features_with_constant.astype(float)).fit().pvalues[1:]
        max_p_value = p_values.max()
        if(max_p_value >= threshold):
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break 
        print(p_values)
    return features

In [3]:
#%% FUNCTION FOR CLUSTERING

def clustering(df):
    df=df.fillna(df.mean())
    le = LabelEncoder()
    cat_cols=list(df.select_dtypes('object').columns)
    for fet in cat_cols:
        #le.fit(df[fet])
        df[fet] = le.fit_transform(df[fet].astype(str))
    initial_features = df.columns.tolist()
    target=input('ENTER THE TARGET VARIABLE : ')
    Target=df[target]
    X=df.drop(columns=target)
    from varclushi import VarClusHi
    threshold=float(input("Enter eigenvalue : (range 0.1 to 1.0)"))
    var_model=VarClusHi(X,maxeigval2 =threshold,maxclus=None) ##I used maxeigval = 0.7. It means that clusters will split if the second eigenvalue is greater than 0.7. A larger value of this parameter gives fewer clusters and less of the variations explained. Smaller value gives more clusters and more variation explained. The common choice is 1 as it represents the average size of eigenvalues.
    var_model.varclus()
    print(var_model.rsquare)